<a href="https://colab.research.google.com/github/norbchaar/CABD/blob/main/TP3/CABD_TP3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Actualizamos los repositorios
!apt-get update -qq

# Instalamos Spark para Python
!pip install pyspark

# Instalamos Java 8 (versión ligera sin GUI)
!apt-get install -y openjdk-8-jdk-headless -qq > /dev/null

# Configuramos JAVA_HOME
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# Seleccionamos Java 8 (si es necesario)
!echo 2 | update-alternatives --config java

# Validamos instalación de Java
!java -version

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
There are 2 choices for the alternative java (providing /usr/bin/java).

  Selection    Path                                            Priority   Status
------------------------------------------------------------
  0            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      auto mode
  1            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      manual mode
* 2            /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java   1081      manual mode

Press <enter> to keep the current choice[*], or type selection number: openjdk version "1.8.0_432"
OpenJDK Runtime Environment (build 1.8.0_432-8u432-ga~us1-0ubuntu2~22.04-ga)
OpenJDK 64-Bit Server VM (build 25.432-bga, mixed mode)


In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.streaming import StreamingContext
import os, time, shutil
import threading

# "VALOR DE ANTIGUEDAD"
A = .5

# Configuración de rutas
root_path = '/content/'
input_path = root_path + "input"  # Carpeta original con los archivos
input_streaming = root_path + "input_streaming"  # Carpeta monitoreada por Spark
buffer = root_path + "buffer"

# Preparar carpetas
if not os.path.exists(input_streaming):
    os.makedirs(input_streaming)
else:
    for f in os.listdir(input_streaming):
        os.remove(os.path.join(input_streaming, f))

# Configuración de SparkContext y StreamingContext
sc = SparkContext("local[2]", "CABD-TP3")
ssc = StreamingContext(sc, 6)  # Intervalo de microbatch de 6 segundos

# Leer el stream de datos desde `input_streaming`
stream = ssc.textFileStream(input_streaming)




# Transformaciones: Cada línea debe ser del formato <id_user, id_mission, tiempo_juego>
# Separar campos por tabulación
parsed_stream = stream.map(lambda line: line.split("\t"))

# Mapeamos a RDD pareado (id_mission, (tiempo_juego, ocurrencia))
mapped_stream = parsed_stream.map(lambda fields: (int(fields[1]), (float(fields[2]), 1)))

# Acumular tiempos y ocurrencias por misión
accumulations = mapped_stream.reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))

# Calcular los promedios
averages = accumulations.mapValues(lambda x: x[0] / x[1])




# Persistencia entre ventanas
ssc.checkpoint(buffer)

# Actualización entre ventanas
def fUpdate(newValues, scoreAnterior):
    # Si el estado es None, inicializa en 0. Si no, aplica el descuento
    # de antigüedad (evitando valores negativos)
    scoreAnterior = 0 if scoreAnterior is None else max(scoreAnterior - A, 0)
    # Suma los valores nuevos si existen
    newValuesSum = sum(newValues) if newValues else 0
    # Actualiza el score
    updatedScore = scoreAnterior + newValuesSum
    # Si el estado es 0, devuelve None para evitar guardar una clave innecesaria
    return None if updatedScore == 0 else updatedScore

newScores = averages.updateStateByKey(fUpdate)
newScores.pprint()




# Simulación de la llegada de archivos
def simulate_file_arrival():
    for file_name in sorted(os.listdir(input_path)):  # Ordena los archivos alfabéticamente
        source = os.path.join(input_path, file_name)
        destination = os.path.join(input_streaming, file_name)
        shutil.copy(source, destination)  # Copiar archivo al directorio monitoreado
        print(f"Archivo {file_name} recibido en {input_streaming}")
        time.sleep(3)  # Simular llegada cada 3 segundos

# Iniciar el streaming
ssc.start()

# Iniciar el hilo de simulación de llegada de archivos
file_thread = threading.Thread(target=simulate_file_arrival)
file_thread.start()

# Ejecución contínua de Spark Streaming
ssc.awaitTermination()


/usr/local/lib/python3.10/dist-packages/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(


Archivo heroes0 recibido en /content/input_streaming
Archivo heroes1 recibido en /content/input_streaming
Archivo heroes10 recibido en /content/input_streaming
Archivo heroes11 recibido en /content/input_streaming
-------------------------------------------
Time: 2024-12-16 02:42:54
-------------------------------------------
(22, 448.0)
(6, 367.0)
(10, 402.0)
(14, 304.0)
(2, 414.6666666666667)
(8, 420.6666666666667)
(26, 317.0)
(16, 389.0)
(20, 376.0)
(4, 417.0)
...

Archivo heroes12 recibido en /content/input_streaming
-------------------------------------------
Time: 2024-12-16 02:43:00
-------------------------------------------
(22, 860.875)
(6, 688.0)
(10, 760.5)
(14, 711.3571428571429)
(2, 811.8333333333334)
(8, 764.1666666666667)
(26, 733.5)
(16, 761.5)
(20, 711.5)
(4, 849.5)
...

Archivo heroes13 recibido en /content/input_streaming
-------------------------------------------
Time: 2024-12-16 02:43:06
-------------------------------------------
(22, 1261.375)
(6, 1098.25)
(10,

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
RuntimeError: reentrant call inside <_io.BufferedReader name=45>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/sock

Py4JError: An error occurred while calling o25.awaitTermination